In [ ]:
import numpy as np
import os
import pandas as pd
import neurokit2 as nk
import glob

In [ ]:
#code to downsample physiology data to 200Hz
# set directory containing CSV files

directory = '/content/drive/MyDrive/scenario_2/fold_0/train/Clean/physiology/'
# directory = '/content/drive/MyDrive/scenario_4/fold_0/Clean/physiology/'
csvname = os.listdir(directory)

for j in range(len(csvname)):
    old_df = pd.read_csv(directory + csvname[j])   # replace '...' with your data and column names
    #find an array with entried like 0,5,10,15,...
    array=range(0, len(old_df),5)
    #take only these index value in old file data
    old_df=old_df.iloc[array]
    new_df=old_df.reset_index(drop=True)
    #save new 200 Hz file 
    new_df.to_csv("/content/drive/MyDrive/scenario_4/fold_0/Clean/200Hz/physiology/" + csvname[j])

In [ ]:
#--------------code to expand anotation file-------------
import glob
# set directory containing CSV files
directory = '/content/drive/MyDrive/scenario_4/fold_0/train/annotations/'

# use glob to get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, '*.csv'))
csvname = os.listdir(directory)

tmp = 0
for csv in csv_files:
  df = pd.read_csv(csv)
# create a new dataframe with interpolated values for desired time intervals
  new_df = pd.DataFrame(columns=['time', 'valence',	'arousal'])
  k = 0
  rate=5 #rates tells into how many parts 50 seconds data should be divided. 
  for i in range(0, max(df['time']), 50):
    k = i
    new_row = pd.DataFrame({'time': range(i, i+50, rate), 'valence': df.loc[df['time'] == i, 'valence'].values[0], 'arousal': df.loc[df['time'] == i, 'arousal'].values[0]})
    new_df = pd.concat([new_df, new_row], ignore_index=True)
    
  new_row = pd.DataFrame({'time': range(k+50 ,k+51), 'valence': df.loc[df['time'] == k + 50, 'valence'].values[0], 'arousal': df.loc[df['time'] == k+50, 'arousal'].values[0]})
  new_df =  pd.concat([new_df, new_row], ignore_index=True)
  #store in a new directory 
  storeat = "/content/drive/MyDrive/scenario_4/fold_0/Clean/200Hz/annotations/" + csvname[tmp]

  tmp = tmp + 1
  # write new dataframe to csv
  new_df.to_csv(storeat, index=False)

In [ ]:
#----------------------------------------------------------------------
# code to apply 2 second time delay window between physiolocial and annotation data and then merge the data
# apply a time window of 2seconds and then merge data 
# for physio data, remove 2 seconds begining data. for annotation data, remove 3 seonds anno data
#in below line, define path of folder

path= ('/content/drive/MyDrive/scenario_4/fold_0/Clean/200Hz/physiology/')

# enter subjects and video numbers. This is done to make sure merging happends in same way in both physiology and annotation
sub= [1,2,3,5,6,8,9,10,11,12,16,18,19,20,21,24,25,27,29,30,32,33,34,35,36,38,40,43,44] #for scenerio 1: : Do not change this line
vid=[3,16,19,20] #for scenerio 1: : Do not change this line

# sub= [1,4,6,7,8,9,11,12,13,14,17,18,19,20,22,26,28,29,30,31,32,33,34,35,36,37,38,41,43,45] #for scenerio 1: : Do not change this line
# vid=[1,9,10,11,13,14,18,20] #for scenerio 1: : Do not change this line
# sub= [0,3,4,5,8,10,11,13,14,15,16,17,18,19,21,22,23,26,30,32,33,34,35,36,39,40,41,42,43,44]# for scenerio 3 fold 0: Do not change this line
# vid=[0,3,4,10,21,22] # for scenerio 3 fold 0: Do not change this line
# sub= [0,1,2,5,7,9,11,13,15,17,18,20,24,26,27,28,29,30,31,32,34,41,43,45] # for scenerio 2 fold 0: Do not change this line
# vid=[0,2,9,10,11,13,14,20] # for scenerio 2 fold 0: Do not change this line

sub2=[]
vid2= []

mergedFile=pd.DataFrame()
for sub in sub:
   for X in vid:
    name = f'sub_{sub}_vid_{X}.csv.csv'   
    temp = pd.read_csv(path+ name)
    # in below lines are commented, then no time delay would be applied. Uncomment two lines depending on if you are working with physiological or annotation data
    # #the below lines drop 2 seconds data at the begining of annotaion file
    # temp=temp.drop(temp.index[:400])
    # #the below lines drop 2 seconds data at the end of physiology file
    # temp = temp.drop(temp.index[-400:])
    mergedFile=mergedFile.append(temp,ignore_index=True)

# save files in a folder
path2=f'/content/drive/MyDrive/scenario_4/fold_0/Clean/200Hz/merged/mergeddataphysio.csv'
mergedFile.to_csv(path2)